Importación y Carga de Datos

In [7]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras_tuner import Hyperband
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('telecom_customer_churn.csv')

Preprocesamiento

In [8]:
# 1. Preprocesamiento
# a. Eliminar entradas con el estado "Joined"
df = df[df['Customer Status'] != 'Joined']

# b. Crear la columna 'Churn' y asignar 1 si 'Customer Status' es 'Churned', de lo contrario 0
df['Churn'] = df['Customer Status'].apply(lambda x: 1 if x == 'Churned' else 0)

# c. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# d. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# e. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Customer Status':
        df[column].fillna(df[column].mode()[0], inplace=True)
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)

# Eliminar la columna 'Customer Status' antes de normalizar
df = df.drop(columns=['Customer Status'])

# f. Normalización
cols_to_scale = df.columns.difference(['Churn'])
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

Selección de características - Gradient Boosting

Balanceo con SMOTE

In [9]:
# 3. Balanceo con SMOTE
X = df.drop('Churn', axis=1)
y = df['Churn']
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

División de Conjunto

In [10]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

Preparación de Datos para RNN

In [11]:
# 5. Preparación de Datos para RNN
X_train_rnn = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

Entrenamiento de RNN

In [12]:
# 6. Entrenamiento de RNN con búsqueda de hiperparámetros usando Keras Tuner
def build_rnn_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=8, max_value=64, step=8), 
                   input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('units', min_value=8, max_value=64, step=8), return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    # Utilizar Adam sin el prefijo legacy
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Continúa con la configuración de Hyperband y la búsqueda de hiperparámetros
tuner_rnn = Hyperband(
    build_rnn_model,
    objective='val_accuracy',
    max_epochs=100,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='keras_tuner_rnn_example'
)

callbacks_rnn = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.ModelCheckpoint('best_rnn_model.keras', save_best_only=True)  # Cambiado de .h5 a .keras
]

tuner_rnn.search(X_train_rnn, y_train, epochs=100, validation_split=0.2, callbacks=callbacks_rnn)

# Obtener los mejores hiperparámetros
best_hps_rnn = tuner_rnn.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Los mejores hiperparámetros encontrados:
Número de unidades: {best_hps_rnn.get('units')}
Tasa de aprendizaje: {best_hps_rnn.get('learning_rate')}
""")

# Obtener el mejor modelo RNN
best_rnn_model = tuner_rnn.get_best_models(num_models=1)[0]

Trial 24 Complete [00h 00m 09s]
val_accuracy: 0.806750476360321

Best val_accuracy So Far: 0.8590337634086609
Total elapsed time: 00h 03m 30s

Los mejores hiperparámetros encontrados:
Número de unidades: 48
Tasa de aprendizaje: 0.01



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Selección de Hiperparámetros

In [13]:
# 7. Evaluación del modelo con los mejores hiperparámetros
loss_rnn, accuracy_rnn = best_rnn_model.evaluate(X_test_rnn, y_test)
print(f'Loss: {loss_rnn}, Accuracy: {accuracy_rnn}')

# Obtener las predicciones en forma de clases
y_pred_rnn = (best_rnn_model.predict(X_test_rnn) > 0.5).astype("int32")

# Calcular la matriz de confusión, y generar el reporte de clasificación
conf_matrix_rnn = confusion_matrix(y_test, y_pred_rnn)
report_rnn = classification_report(y_test, y_pred_rnn)

# Imprimir las métricas
print(conf_matrix_rnn)
print("Reporte de clasificación:\n", report_rnn)

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8423 - loss: 0.3633
Loss: 0.3365612328052521, Accuracy: 0.8559321761131287
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[797 157]
 [115 819]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.87      0.84      0.85       954
           1       0.84      0.88      0.86       934

    accuracy                           0.86      1888
   macro avg       0.86      0.86      0.86      1888
weighted avg       0.86      0.86      0.86      1888

